In [46]:
import pandas as pd
import numpy as np
import math
import math
from pathlib import Path

from tournament import Tournament, Participant, Match

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"
GAME_ID = 406  #4 Team 2021
GAME_ID = 359  # 6 Team 2016
# GAME_ID = 273  # 8 team 2012

In [47]:
league_settings_query = f"SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week from dev.league_settings where game_id = '{GAME_ID}'"
league_settings = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    league_settings_query
).drop_duplicates()

meta_query = "SELECT game_id, league_id, end_week FROM dev.league_metadata"
meta = DatabaseCursor(PATH).copy_data_from_postgres(meta_query).drop_duplicates()

league_settings = league_settings.merge(meta, how='inner', left_on=['game_id', 'league_id'], right_on=['game_id', 'league_id'])

num_playoff_teams = league_settings["num_playoff_teams"].values[0]
playoff_start_week = league_settings["playoff_start_week"].values[0]
playoff_end_week = league_settings["end_week"].values[0]
playoff_weeks = range(playoff_start_week, playoff_end_week+1)

matchup_board_query = f"SELECT * FROM prod.matchup_board_{int(GAME_ID)}"
matchup_board = DatabaseCursor(PATH, options=OPTION_PROD).copy_data_from_postgres(
    matchup_board_query
).drop_duplicates()

playoff_teams = list(
    matchup_board["team_key"][matchup_board["reg_season_rank"] <= num_playoff_teams]
)

playoff_teams_seeds = matchup_board[["team_key", 'reg_season_rank']][matchup_board["reg_season_rank"] <= num_playoff_teams]

matchup_board.set_index(matchup_board['team_key'], drop=True, inplace=True)

Successfully pulled: SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week from dev.league_settings where game_id = '359'
Successfully pulled: SELECT game_id, league_id, end_week FROM dev.league_metadata
Successfully pulled: SELECT * FROM prod.matchup_board_359


In [48]:
matchup_board

,game_id,reg_season_rank,team_key,team,mgr,2pt_rank,2pt_total,w_l_rank,wins,losses,pts_rank,pts_ttl,pro_pts_rank,pro_pts_ttl,week_1,pts_1,pro_pts_1,opp_key_1,opp_team_1,opp_mgr_1,opp_pts_1,opp_pro_pts_1,win_loss_1,2pt_score_1,week_2,pts_2,pro_pts_2,opp_key_2,opp_team_2,opp_mgr_2,opp_pts_2,opp_pro_pts_2,win_loss_2,2pt_score_2,week_3,pts_3,pro_pts_3,opp_key_3,opp_team_3,opp_mgr_3,opp_pts_3,opp_pro_pts_3,win_loss_3,2pt_score_3,week_4,pts_4,pro_pts_4,opp_key_4,opp_team_4,opp_mgr_4,opp_pts_4,opp_pro_pts_4,win_loss_4,2pt_score_4,week_5,pts_5,pro_pts_5,opp_key_5,opp_team_5,opp_mgr_5,opp_pts_5,opp_pro_pts_5,win_loss_5,2pt_score_5,week_6,pts_6,pro_pts_6,opp_key_6,opp_team_6,opp_mgr_6,opp_pts_6,opp_pro_pts_6,win_loss_6,2pt_score_6,week_7,pts_7,pro_pts_7,opp_key_7,opp_team_7,opp_mgr_7,opp_pts_7,opp_pro_pts_7,win_loss_7,2pt_score_7,week_8,pts_8,pro_pts_8,opp_key_8,opp_team_8,opp_mgr_8,opp_pts_8,opp_pro_pts_8,win_loss_8,2pt_score_8,week_9,pts_9,pro_pts_9,opp_key_9,opp_team_9,opp_mgr_9,opp_pts_9,opp_pro_pts_9,win_loss_9,2pt_score_9,week_10,pts_10,pro_pts_10,opp_key_10,opp_team_10,opp_mgr_10,opp_pts_10,opp_pro_pts_10,win_loss_10,2pt_score_10,week_11,pts_11,pro_pts_11,opp_key_11,opp_team_11,opp_mgr_11,opp_pts_11,opp_pro_pts_11,win_loss_11,2pt_score_11,week_12,pts_12,pro_pts_12,opp_key_12,opp_team_12,opp_mgr_12,opp_pts_12,opp_pro_pts_12,win_loss_12,2pt_score_12,week_13,pts_13,pro_pts_13,opp_key_13,opp_team_13,opp_mgr_13,opp_pts_13,opp_pro_pts_13,win_loss_13,2pt_score_13,week_14,pts_14,pro_pts_14,week_15,pts_15,pro_pts_15,week_16,pts_16,pro_pts_16
team_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
359.l.185373.t.5,359,1,359.l.185373.t.5,Lock Jaw Blowy,Pete,2,16,1,8,5,2,1811.46,1,1850.69,1,112.66,105.22,359.l.185373.t.7,Push Ups in Prison,Pat,124.66,108.68,L,0,2,111.52,102.34,359.l.185373.t.8,Breesus Christ,Wes,123.32,109.97,L,1,3,94.00,107.68,359.l.185373.t.9,Brady didn't do it,Jake,89.04,114.75,W,0,4,123.74,117.63,359.l.185373.t.1,2 Gurley's 1 Cup,Tim,101.44,89.27,W,1,5,124.86,121.05,359.l.185373.t.2,Adele Beckham Jr.,James,108.58,102.18,W,1,6,89.66,125.76,359.l.185373.t.3,Arian Nation,Ryan,141.50,116.83,L,0,7,144.74,117.58,359.l.185373.t.4,Stairway to Evans,Greg,128.88,104.02,W,1,8,110.94,104.24,359.l.185373.t.10,Dez Nuts,Jeremy,117.22,101.67,L,1,9,82.98,119.76,359.l.185373.t.6,Deckers Barn animals,Kevin,77.76,105.82,W,0,10,130.44,116.20,359.l.185373.t.7,Push Ups in Prison,Pat,116.44,118.93,W,1,11,134.44,125.94,359.l.185373.t.8,Breesus Christ,Wes,98.30,102.34,W,1,12,109.42,114.99,359.l.185373.t.9,Brady didn't do it,Jake,94.90,102.19,W,1,13,90.96,123.09,359.l.185373.t.1,2 Gurley's 1 Cup,Tim,103.38,106.93,L,0,14,134.24,117.92,15,85.58,114.95,16,131.28,116.34
359.l.185373.t.7,359,2,359.l.185373.t.7,Push Ups in Prison,Pat,1,17,1,8,5,6,1655.76,2,1822.22,1,124.66,108.68,359.l.185373.t.5,Lock Jaw Blowy,Pete,112.66,105.22,W,1,2,121.12,117.60,359.l.185373.t.6,Deckers Barn animals,Kevin,110.76,109.33,W,1,3,102.78,120.08,359.l.185373.t.10,Dez Nuts,Jeremy,149.08,115.31,L,1,4,86.20,125.86,359.l.185373.t.8,Breesus Christ,Wes,110.48,109.46,L,0,5,97.24,112.06,359.l.185373.t.9,Brady didn't do it,Jake,81.38,112.88,W,0,6,104.98,123.00,359.l.185373.t.1,2 Gurley's 1 Cup,Tim,72.36,106.18,W,1,7,122.34,111.23,359.l.185373.t.2,Adele Beckham Jr.,James,66.72,112.31,W,1,8,118.28,114.92,359.l.185373.t.3,Arian Nation,Ryan,102.60,101.37,W,1,9,131.90,106.71,359.l.185373.t.4,Stairway to Evans,Greg,118.14,117.91,W,1,10,116.44,118.93,359.l.185373.t.5,Lock Jaw Blowy,Pete,130.44,116.20,L,1,11,70.58,101.48,359.l.185373.t.6,Deckers Barn animals,Kevin,122.10,101.09,L,0,12,78.14,116.12,359.l.185373.t.10,Dez Nuts,Jeremy,117.54,113.53,L,0,13,114.58,112.86,359.l.185373.t.8,Breesus Christ,Wes,78.64,101.06,W,1,14,40.80,115.86,15,122.20,106.33,16,103.52,110.50
359.l.185373.t.8,359,3,359.l.185373.t.8,Breesus Christ,Wes,3,15,1,8,5,9,1562.62,6,1717.74,1,94.42,111.21,359.l.185373.t.4

In [152]:
next_higher_power_of_two = int(math.pow(2, math.ceil(math.log2(len(playoff_teams)))))
print(next_higher_power_of_two)
winners_number_of_byes = next_higher_power_of_two - len(playoff_teams)
print(winners_number_of_byes)
incoming_participants = list(playoff_teams)
print(incoming_participants)
incoming_participants.extend(["Bye"] * winners_number_of_byes)
print(incoming_participants)
num_of_rounds = len(playoff_weeks)
print(num_of_rounds)
num_of_matches = int((len(incoming_participants) / 2) * num_of_rounds)
print(num_of_matches)
matches_per_round = int(num_of_matches / num_of_rounds)
print(matches_per_round)
half_length = int(len(incoming_participants) / 2)
print(half_length)
first = incoming_participants[0:half_length]
print(first)
last = incoming_participants[half_length:]
last.reverse()
print(last)

8
0
['273.l.777818.t.7', '273.l.777818.t.10', '273.l.777818.t.2', '273.l.777818.t.6', '273.l.777818.t.4', '273.l.777818.t.9', '273.l.777818.t.12', '273.l.777818.t.1']
['273.l.777818.t.7', '273.l.777818.t.10', '273.l.777818.t.2', '273.l.777818.t.6', '273.l.777818.t.4', '273.l.777818.t.9', '273.l.777818.t.12', '273.l.777818.t.1']
3
12
4
4
['273.l.777818.t.7', '273.l.777818.t.10', '273.l.777818.t.2', '273.l.777818.t.6']
['273.l.777818.t.1', '273.l.777818.t.12', '273.l.777818.t.9', '273.l.777818.t.4']


In [5]:
def tournament_round(df, week, teams):

    tourney = Round(teams)
    winner_list = []
    loser_list = []
    match_list = []
    for match in tourney.get_active_matches():
        home = match.get_participants()[0].get_competitor()
        home_score = df[f'pts_{week}'][df['team_key'] == home].values[0]
        away = match.get_participants()[1].get_competitor()
        away_score = df[f'pts_{week}'][df['team_key'] == away].values[0]
        match_list.append([home, away])

        if home_score > away_score:
            winner_list.append(home)
            loser_list.append(away)
        elif home_score < away_score:
            winner_list.append(away)
            loser_list.append(home)
        else:
            print('Error')
    
    return winner_list, loser_list, match_list

def results(contestants):
   
    for i in range(len(contestants)):
        lossing_team = matchups[i][1] if winners[i] == matchups[i][0] else matchups[i][0]
        winning_team = matchups[i][1] if losers[i] == matchups[i][0] else matchups[i][0]

        matchup_board.at[winning_team, f'opp_key_{wk}'] = lossing_team
        matchup_board.at[winning_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == lossing_team].values[0]
        matchup_board.at[winning_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == lossing_team].values[0]
        matchup_board.at[winning_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
        matchup_board.at[winning_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
        matchup_board.at[winning_team, f'win_loss_{wk}'] = 'W'
        matchup_board.at[winning_team, f'consol_{wk}'] = 'N'

        matchup_board.at[lossing_team, f'opp_key_{wk}'] = winning_team
        matchup_board.at[lossing_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == winning_team].values[0]
        matchup_board.at[lossing_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == winning_team].values[0]
        matchup_board.at[lossing_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
        matchup_board.at[lossing_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
        matchup_board.at[lossing_team, f'win_loss_{wk}'] = 'L'
        matchup_board.at[lossing_team, f'consol_{wk}'] = 'Y'


In [6]:
for wk in playoff_weeks:
    if wk == playoff_start_week:
        teams = list(matchup_board["team_key"][matchup_board["reg_season_rank"] <= num_playoff_teams])
        winners, losers, matchups = tournament_round(matchup_board, wk, teams)

        for i in range(len(winners)):
            lossing_team = matchups[i][1] if winners[i] == matchups[i][0] else matchups[i][0]
            winning_team = matchups[i][1] if losers[i] == matchups[i][0] else matchups[i][0]

            matchup_board.at[winning_team, f'opp_key_{wk}'] = lossing_team
            matchup_board.at[winning_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'win_loss_{wk}'] = f'W.{wk}.{i}'

            matchup_board.at[lossing_team, f'opp_key_{wk}'] = winning_team
            matchup_board.at[lossing_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'win_loss_{wk}'] = f'L.{wk}.{i}'

    elif wk == playoff_start_week+1:
        matchup_board.sort_values([f'win_loss_{wk-1}'], inplace=True)
        p_teams = list(matchup_board["team_key"][matchup_board[f"win_loss_{wk-1}"].str.contains('W') == True])
        c_teams = list(matchup_board["team_key"][matchup_board[f"win_loss_{wk-1}"].str.contains('L') == True])
        p_winners, p_losers, p_matchups = tournament_round(matchup_board, wk, p_teams)
        c_winners, c_losers, c_matchups = tournament_round(matchup_board, wk, c_teams)
        print(p_winners, p_losers, p_matchups, sep='\n')
        print(c_winners, c_losers, c_matchups, sep='\n')

        for i in range(len(p_winners)):
            lossing_team = p_matchups[i][1] if p_winners[i] == p_matchups[i][0] else p_matchups[i][0]
            winning_team = p_matchups[i][1] if p_losers[i] == p_matchups[i][0] else p_matchups[i][0]

            matchup_board.at[winning_team, f'opp_key_{wk}'] = lossing_team
            matchup_board.at[winning_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'win_loss_{wk}'] = f'W.{wk}.{i}'

            matchup_board.at[lossing_team, f'opp_key_{wk}'] = winning_team
            matchup_board.at[lossing_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'win_loss_{wk}'] = f'L.{wk}.{i}'

        for i in range(len(c_winners)):
            lossing_team = c_matchups[i][1] if c_winners[i] == c_matchups[i][0] else c_matchups[i][0]
            winning_team = c_matchups[i][1] if c_losers[i] == c_matchups[i][0] else c_matchups[i][0]

            matchup_board.at[winning_team, f'opp_key_{wk}'] = lossing_team
            matchup_board.at[winning_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
            matchup_board.at[winning_team, f'win_loss_{wk}'] = f'C.W.{wk}.{i}'

            matchup_board.at[lossing_team, f'opp_key_{wk}'] = winning_team
            matchup_board.at[lossing_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
            matchup_board.at[lossing_team, f'win_loss_{wk}'] = f'C.L.{wk}.{i}'



['273.l.777818.t.6', '273.l.777818.t.12']
['273.l.777818.t.7', '273.l.777818.t.2']
[['273.l.777818.t.7', '273.l.777818.t.6'], ['273.l.777818.t.12', '273.l.777818.t.2']]
['273.l.777818.t.1', '273.l.777818.t.10']
['273.l.777818.t.4', '273.l.777818.t.9']
[['273.l.777818.t.1', '273.l.777818.t.4'], ['273.l.777818.t.10', '273.l.777818.t.9']]


In [7]:
cols_15 = ['opp_key_15', 'reg_season_rank', 'team', 'mgr', 
'week_15', 'pts_15', 'pro_pts_15','opp_team_15', 'opp_mgr_15', 'opp_pts_15', 'opp_pro_pts_15', 'win_loss_15']
matchup_board[cols_15][matchup_board['reg_season_rank'] <= 8]

,opp_key_15,reg_season_rank,team,mgr,week_15,pts_15,pro_pts_15,opp_team_15,opp_mgr_15,opp_pts_15,opp_pro_pts_15,win_loss_15
team_key,,,,,,,,,,,,
273.l.777818.t.1,273.l.777818.t.7,8,Taco,James,15,65.86,102.73,Goons,Jake,88.50,106.62,L.15.0
273.l.777818.t.10,273.l.777818.t.12,2,Tebowners,273.l.777818.t.10,15,106.34,117.06,ghost,273.l.777818.t.12,131.12,96.76,L.15.1
273.l.777818.t.9,273.l.777818.t.2,6,Ruxin,273.l.777818.t.9,15,63.68,110.55,Bro Lo El Cunado,273.l.777818.t.2,71.90,58.54,L.15.2
273.l.777818.t.4,273.l.777818.t.6,5,The Fear Boners,Wes,15,75.38,85.51,ELE,Pete,77.14,90.89,L.15.3
273.l.777818.t.7,273.l.777818.t.1,1,Goons,Jake,15,88.50,106.62,Taco,James,65.86,102.73,W.15.0
273.l.777818.t.12,273.l.777818.t.10,7,ghost,273.l.777818.t.12,15,131.12,96.76,Tebowners,273.l.777818.t.10,106.34,117.06,W.15.1
273.l.777818.t.2,273.l.777818.t.9,3,Bro Lo El Cunado,273.l.777818.t.2,15,71.90,58.54,Ruxin,273.l.777818.t.9,63.68,110.55,W.15.2
273.l.777818.t.6,273.l.777818.t.4,4,ELE,Pete,15,77.14,90.89,The Fear Boners,Wes,75.38,85.51,W.15.3


In [8]:
cols_16 = ['opp_key_16','game_id', 'reg_season_rank', 'team', 'mgr', 
'week_16', 'pts_16', 'pro_pts_16', 'opp_team_16', 'opp_mgr_16', 'opp_pts_16', 'opp_pro_pts_16', 'win_loss_16']
matchup_board.sort_values([f'win_loss_{16}'], inplace=True)
matchup_board[cols_16][matchup_board['reg_season_rank'] <= 8]

,opp_key_16,game_id,reg_season_rank,team,mgr,week_16,pts_16,pro_pts_16,opp_team_16,opp_mgr_16,opp_pts_16,opp_pro_pts_16,win_loss_16
team_key,,,,,,,,,,,,,
273.l.777818.t.4,273.l.777818.t.1,273,5,The Fear Boners,Wes,16,120.76,93.67,Taco,James,135.44,96.65,C.L.16.0
273.l.777818.t.9,273.l.777818.t.10,273,6,Ruxin,273.l.777818.t.9,16,90.34,103.94,Tebowners,273.l.777818.t.10,91.20,123.18,C.L.16.1
273.l.777818.t.1,273.l.777818.t.4,273,8,Taco,James,16,135.44,96.65,The Fear Boners,Wes,120.76,93.67,C.W.16.0
273.l.777818.t.10,273.l.777818.t.9,273,2,Tebowners,273.l.777818.t.10,16,91.20,123.18,Ruxin,273.l.777818.t.9,90.34,103.94,C.W.16.1
273.l.777818.t.7,273.l.777818.t.6,273,1,Goons,Jake,16,111.66,96.35,ELE,Pete,123.50,96.32,L.16.0
273.l.777818.t.2,273.l.777818.t.12,273,3,Bro Lo El Cunado,273.l.777818.t.2,16,69.02,71.45,ghost,273.l.777818.t.12,110.48,96.40,L.16.1
273.l.777818.t.6,273.l.777818.t.7,273,4,ELE,Pete,16,123.50,96.32,Goons,Jake,111.66,96.35,W.16.0
273.l.777818.t.12,273.l.777818.t.2,273,7,ghost,273.l.777818.t.12,16,110.48,96.40,Bro Lo El Cunado,273.l.777818.t.2,69.02,71.45,W.16.1


In [49]:
competitors_list = list(zip(playoff_teams_seeds['team_key'], playoff_teams_seeds['reg_season_rank']))
competitors_list

[('359.l.185373.t.5', 1),
 ('359.l.185373.t.7', 2),
 ('359.l.185373.t.8', 3),
 ('359.l.185373.t.4', 4),
 ('359.l.185373.t.10', 5),
 ('359.l.185373.t.1', 6)]

In [56]:
__matches = []
next_higher_power_of_two = int(
    math.pow(2, math.ceil(math.log2(len(competitors_list))))
)
winners_number_of_byes = next_higher_power_of_two - len(competitors_list)
competitors_list.extend(['Bye'] * winners_number_of_byes)
incoming_participants = list(map(Participant, competitors_list))
num_of_rounds = int(math.ceil(math.log2(len(incoming_participants))))

round_ = 1
while round_ <= num_of_rounds:
    if round_ == 1:
        print('1: ', incoming_participants)
        half_length = int(len(incoming_participants) / 2)
        first = incoming_participants[0:half_length]
        last = incoming_participants[half_length:]
        last.reverse()
        round_1_winners = []
        round_1_losers = []
        for participant_pair in zip(first, last):
            if participant_pair[1] is None:
                round_1_winners.append(participant_pair[0])
            elif participant_pair[0] is None:
                round_1_winners.append(participant_pair[1])
            else:
                match = Match(participant_pair[0], participant_pair[1])
                round_1_winners.append(match.get_winner_participant())
                round_1_losers.append(match.get_loser_participant())
                __matches.append(match)
        next_round = [round_1_winners, round_1_losers]
        print('2: ', next_round)
        round_ += 1
    
    elif round_ > 1:
        while True:
            if len(next_round[0]) > 1:
                bracket = next_round[0]
                print(f'{next_round.index(bracket)}: ', bracket)
                half_length = int(len(bracket) / 2)
                first = bracket[0:half_length]
                last = bracket[half_length:]
                last.reverse()
                next_round_winners = []
                next_round_losers = []
                for participant_pair in zip(first, last):
                    if participant_pair[1] is None:
                        next_round_winners.append(participant_pair[0])
                    elif participant_pair[0] is None:
                        next_round_winners.append(participant_pair[1])
                    else:
                        match = Match(participant_pair[0], participant_pair[1])
                        next_round_winners.append(match.get_winner_participant())
                        print('Winner: ', match.get_winner_participant())
                        next_round_losers.append(match.get_loser_participant())
                        print('Loser: ', match.get_loser_participant())
                        __matches.append(match)
                try:
                    next_round.remove(bracket)
                except:
                    continue
                next_round.append(next_round_winners)
                next_round.append(next_round_losers)
                next_round = [ele for ele in next_round if ele != []]
                print('NR: ', next_round)
                if len(next_round) == len(incoming_participants):
                    break
            round_ += 1

1:  [<tournament.Participant object at 0x7f850b162080>, <tournament.Participant object at 0x7f850b162140>, <tournament.Participant object at 0x7f850b162170>, <tournament.Participant object at 0x7f850b162260>, <tournament.Participant object at 0x7f850b1622c0>, <tournament.Participant object at 0x7f850b163610>, <tournament.Participant object at 0x7f850b161ed0>, <tournament.Participant object at 0x7f850b161e10>]
2:  [[<tournament.Participant object at 0x7f850b5bea10>, <tournament.Participant object at 0x7f850b5bf3a0>, <tournament.Participant object at 0x7f850b5bea40>, <tournament.Participant object at 0x7f850b5bd5d0>], [<tournament.Participant object at 0x7f850b5be4d0>, <tournament.Participant object at 0x7f850b5bffd0>, <tournament.Participant object at 0x7f850b5bda80>, <tournament.Participant object at 0x7f850b5bdc60>]]
0:  [<tournament.Participant object at 0x7f850b5bea10>, <tournament.Participant object at 0x7f850b5bf3a0>, <tournament.Participant object at 0x7f850b5bea40>, <tournament.